# ☁️ Cloud FinOps CUD Analysis Platform
## Complete Standalone Solution with GCS Integration

**Author:** andrewanolasco@
**Version:** V1.0.0
**Date:** August 2025

---

### Overview
This notebook demonstrates how to use the `finops-analysis-platform` library to analyze Google Cloud Committed Use Discounts (CUDs). The library provides a modular and extensible framework for performing sophisticated FinOps analysis.

## ⚠️ Important: Setup Instructions

1.  **Run Setup Script**: Before running this notebook, open a terminal, navigate to the project's root directory (`cud_analysis`), and run the setup script: `bash scripts/setup_gcp_notebook.sh`.
2.  **Set Working Directory**: The first code cell in this notebook changes the directory to the project root. **You must run it before any other cell.**
3.  **Restart Kernel**: After running the `%cd ..` command, **you must restart the kernel** for the changes to take effect. This will ensure that the notebook can find the project files and configuration.

In [ ]:
# This command sets the notebook's working directory to the project root.
# Please run this cell first, and then RESTART the kernel.
%cd ..

## 📚 Step 1: Import Necessary Modules

In [ ]:
import logging
from finops_analysis_platform.config_manager import ConfigManager
from finops_analysis_platform.data_loader import GCSDataLoader
from finops_analysis_platform.core import CUDAnalyzer
from finops_analysis_platform.reporting import create_dashboard, PDFReportGenerator

# Set up basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## ⚙️ Step 2: Configuration

The `ConfigManager` loads configuration from `config.yaml` and can be overridden by environment variables. Make sure your `config.yaml` is pointing to the correct GCS bucket.

### Required GCS Structure
For the tool to work correctly, your data should be organized in your GCS bucket as follows:
```
gs://your-bucket-name/
├── data/billing/           # Billing export CSVs
├── data/recommendations/   # Cost recommender exports (optional)
└── data/manual_analysis/   # Manual analysis files (optional)
```

In [ ]:
config_manager = ConfigManager(config_path='config.yaml')
print("✅ Configuration loaded.")

## 📂 Step 3: Load Data

The `GCSDataLoader` fetches billing and other data from your configured GCS bucket. If it can't connect, it will generate sample data for demonstration purposes.

In [ ]:
gcs_config = config_manager.get('gcs', {})
loader = GCSDataLoader(bucket_name=gcs_config.get('bucket_name'))
data = loader.load_all_data()
billing_data = data.get('billing')

if 'sample_data' in data:
    print("⚠️ Using sample data for demonstration.")

## 🔬 Step 4: Run the Analysis

The `CUDAnalyzer` performs the core analysis based on the loaded configuration and data.

In [ ]:
analyzer = CUDAnalyzer(config_manager=config_manager, billing_data=billing_data)
analysis = analyzer.generate_comprehensive_analysis()
print("✅ Analysis complete!")

## 📊 Step 5: Generate Reports

Now we can generate a PDF report and an interactive dashboard.

In [ ]:
# Generate PDF Report
if config_manager.get('reporting', {}).get('generate_pdf', True):
    pdf_generator = PDFReportGenerator(config_manager=config_manager)
    report_filename = pdf_generator.generate_report(analysis)
    print(f"📄 PDF report generated: {report_filename}")

# Create Interactive Dashboard
if config_manager.get('reporting', {}).get('create_dashboard', True):
    dashboard = create_dashboard(analysis)
    # In a real notebook environment, this would display the Plotly chart.
    print("📊 Interactive dashboard created.")

## 🕵️ Step 7: Data Profiling (Optional)\n\nYou can also generate a data profiling report to get a deeper understanding of your datasets.

In [ ]:
from finops_analysis_platform.profiler import create_profile_report

# Profile the billing dataset
if 'billing' in data:
    create_profile_report(data['billing'], title='Billing Data Profile')

## 🎉 Conclusion

This notebook has demonstrated the streamlined workflow for running CUD analysis using the refactored library. The process is now more modular, configurable, and easier to maintain.